In [7]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import load_img, img_to_array
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.preprocessing import LabelEncoder
from sqlalchemy import create_engine, Column, Integer, String, LargeBinary
from sqlalchemy.orm import sessionmaker, declarative_base
import os
from sqlalchemy.orm import declarative_base

In [8]:
train_kelias = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\trenyravimas"
val_kelias = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\validacija"
test_kelias = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\testas"

In [9]:

Bazine_klase = declarative_base()
rysys_su_baze = create_engine('sqlite:///pomidoru_lapai.db')

Session = sessionmaker(bind=rysys_su_baze)
sesija = Session()

In [10]:
class PomidoraiTrenyravimas(Bazine_klase):

    __tablename__ = 'Pomidoru_lapai_trenyravimo_duomenys'

    id = Column(Integer,primary_key = True)
    kelias = Column(String, unique=True, nullable=False)  
    klases_pavadinimas = Column(String, nullable=False)


In [11]:
def irasyti_trenyravimo_paveikslelius(sesija, trenyravimo_kelias):

    for klases_pavadinimas in os.listdir(trenyravimo_kelias):
        klases_kelias = os.path.join(trenyravimo_kelias,klases_pavadinimas)

        if os.path.isdir(klases_kelias):
            for paveikslelio_pavadinimas in os.listdir(klases_kelias):
                paveikslelio_kelias = os.path.join(klases_kelias,paveikslelio_pavadinimas)

                egzistuoja = sesija.query(PomidoraiTrenyravimas).filter_by(kelias=paveikslelio_kelias).first()

                if not egzistuoja:

                    naujas_irasas = PomidoraiTrenyravimas(kelias = paveikslelio_kelias, klases_pavadinimas = klases_pavadinimas)

                    sesija.add(naujas_irasas)

    sesija.commit()

    print("Irasyti trenyravimo paveiksleiai")

In [12]:
class PomidoraiValidacija(Bazine_klase):

    __tablename__ = 'Pomidoru_lapai_validacijos_duomenys'

    id = Column(Integer,primary_key = True)
    kelias = Column(String, unique=True, nullable=False)  
    klases_pavadinimas = Column(String, nullable=False)

def irasyti_validacijos_paveikslelius(sesija, validacijos_kelias):

    for klases_pavadinimas in os.listdir(validacijos_kelias):
        klases_kelias = os.path.join(validacijos_kelias,klases_pavadinimas)

        if os.path.isdir(klases_kelias):
            for paveikslelio_pavadinimas in os.listdir(klases_kelias):
                paveikslelio_kelias = os.path.join(klases_kelias,paveikslelio_pavadinimas)

                egzistuoja = sesija.query(PomidoraiValidacija).filter_by(kelias=paveikslelio_kelias).first()

                if not egzistuoja:

                    naujas_irasas = PomidoraiValidacija(kelias = paveikslelio_kelias, klases_pavadinimas = klases_pavadinimas)

                    sesija.add(naujas_irasas)

    sesija.commit()

    print("Irasyti validacijos paveiksleiai")

In [13]:
class PomidoraiTestas(Bazine_klase):

    __tablename__ = 'Pomidoru_lapai_testo_duomenys'

    id = Column(Integer,primary_key = True)
    kelias = Column(String, unique=True, nullable=False)  
    klases_pavadinimas = Column(String, nullable=False)

def irasyti_testo_paveikslelius(sesija, testo_kelias):

    for klases_pavadinimas in os.listdir(test_kelias):
        klases_kelias = os.path.join(test_kelias,klases_pavadinimas)

        if os.path.isdir(klases_kelias):
            for paveikslelio_pavadinimas in os.listdir(klases_kelias):
                paveikslelio_kelias = os.path.join(klases_kelias,paveikslelio_pavadinimas)

                egzistuoja = sesija.query(PomidoraiTestas).filter_by(kelias=paveikslelio_kelias).first()

                if not egzistuoja:

                    naujas_irasas = PomidoraiTestas(kelias = paveikslelio_kelias, klases_pavadinimas = klases_pavadinimas)

                    sesija.add(naujas_irasas)

    sesija.commit()

    print("Irasyti testo paveiksleiai")

In [14]:
Bazine_klase.metadata.create_all(rysys_su_baze)

irasyti_trenyravimo_paveikslelius(sesija,train_kelias)
irasyti_validacijos_paveikslelius(sesija,val_kelias)
irasyti_testo_paveikslelius(sesija,test_kelias)

Irasyti trenyravimo paveiksleiai
Irasyti validacijos paveiksleiai
Irasyti testo paveiksleiai


In [15]:

train_df = pd.read_sql_table('Pomidoru_lapai_trenyravimo_duomenys', con=rysys_su_baze)

val_df = pd.read_sql_table('Pomidoru_lapai_validacijos_duomenys', con=rysys_su_baze)

test_df = pd.read_sql_table('Pomidoru_lapai_testo_duomenys', con=rysys_su_baze)


# print(train_df.head())
# print(val_df.head())
# print(test_df.head())

In [ ]:
def issitraukti_paveikslelius(df, dydis=(128, 128)):
    paveiksleliai = []
    klasifikacijos = []

    for indeksas, eilute in df.iterrows():
        paveikslelis = load_img(eilute['kelias'], target_size=dydis)
        paveikslelis_array = img_to_array(paveikslelis)
        paveiksleliai.append(paveikslelis_array)
        klasifikacijos.append(eilute['klases_pavadinimas'])

    return np.array(paveiksleliai), np.array(klasifikacijos)

x_train, y_train = issitraukti_paveikslelius(train_df)
x_val, y_val = issitraukti_paveikslelius(val_df)
x_test, y_test = issitraukti_paveikslelius(test_df)

In [17]:
def uzkoduoti_klases_lable(y_train, y_val, y_test):

    enkoderis = LabelEncoder()
    y_train_skaiciais = enkoderis.fit_transform(y_train)
    y_val_skaiciais = enkoderis.transform(y_val)
    y_test_skaiciais = enkoderis.transform(y_test)

    print("Klases uzkoduotos")

    return y_train_skaiciais,y_val_skaiciais,y_test_skaiciais

y_train, y_val, y_test = uzkoduoti_klases_lable(y_train,y_val,y_test)

Klases uzkoduotos


Duomenu ikelimas mobile net modeliui 

In [103]:
import tensorflow as tf

paveikslelio_dydis = (224, 224)
imties_dydis = 32
SEED = 42

train_ds = tf.keras.utils.image_dataset_from_directory(
    r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\trenyravimas",
    image_size=paveikslelio_dydis,
    batch_size=imties_dydis,
    label_mode="categorical", 
    shuffle=True,
    seed=SEED)

val_ds = tf.keras.utils.image_dataset_from_directory(
    r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\validacija",
    image_size=paveikslelio_dydis,
    batch_size=imties_dydis,
    label_mode="categorical",
    shuffle=False,
    seed=SEED)

test_ds = tf.keras.utils.image_dataset_from_directory(
    r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\testas",
    image_size=paveikslelio_dydis,
    batch_size=imties_dydis,
    label_mode="categorical",
    shuffle=False,
    seed=SEED)

Found 4500 files belonging to 6 classes.
Found 900 files belonging to 6 classes.
Found 600 files belonging to 6 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import numpy as np
import random
import os

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)


ivesties_forma = (224, 224, 3)
klasiu_skaicius = len(train_ds.class_names)

mobilenet_pagrindas = MobileNetV2(
    input_shape=ivesties_forma,
    include_top=False,
    weights='imagenet')

mobilenet_pagrindas.trainable = False 

modelis = models.Sequential([
    layers.Rescaling(1./255, input_shape=ivesties_forma),
    mobilenet_pagrindas,
    layers.GlobalAveragePooling2D(),
    layers.Dense(180, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(100, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(klasiu_skaicius, activation='softmax')])


modelis.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00008),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

sustabdymas = EarlyStopping(monitor='val_loss',patience=8,restore_best_weights=True)

history = modelis.fit(train_ds,validation_data=val_ds,epochs=50,callbacks=[sustabdymas])


nuostolis, tikslumas = modelis.evaluate(test_ds, verbose=1)

print(f"Testo nuostolis: {nuostolis:4f}")
print(f"Testo tikslumas: {tikslumas:4f}")

Klasiu svoriai
.

In [106]:
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf

dydis = (128, 128)
batch_size = 32
SEED = 42

train_ds = tf.keras.utils.image_dataset_from_directory(
    r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\trenyravimas",
    image_size=(224, 224),
    batch_size=32,
    label_mode="int",
    # label_mode="categorical",
    shuffle=True,
    seed=42)

val_ds = tf.keras.utils.image_dataset_from_directory(
    r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\validacija",
    image_size=(224, 224),
    batch_size=32,
    label_mode="int",
    # label_mode="categorical",
    shuffle=False,
    seed=42)

test_ds = tf.keras.utils.image_dataset_from_directory(
    r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\testas",
    image_size=(224, 224),
    batch_size=32,
    label_mode="int",
    # label_mode="categorical",
    shuffle=False,
    seed=42)

Found 4500 files belonging to 6 classes.
Found 900 files belonging to 6 classes.
Found 600 files belonging to 6 classes.


In [107]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping


ivesties_forma = (224,224, 3)
klasiu_skaicius = len(train_ds.class_names)

mobilenet_pagrindas = MobileNetV2(
    input_shape=ivesties_forma,
    include_top=False,
    weights='imagenet')

mobilenet_pagrindas.trainable = False 

modelis = models.Sequential([
    layers.Rescaling(1./255, input_shape=ivesties_forma),
    mobilenet_pagrindas,
    layers.GlobalAveragePooling2D(),
    layers.Dense(180, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(100, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(klasiu_skaicius, activation='softmax')])


modelis.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00008),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

sustabdymas = EarlyStopping(monitor='val_loss',patience=8,restore_best_weights=True)

c:\Users\Vartotojas\Desktop\POMIDORAI\pomidorai_repositorija\venv\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
klasiu_svoriai = {
    0: 1.0,
    1: 1.0,
    2: 2.0, 
    3: 1.0,
    4: 1.0,
    5: 1.0}

history = modelis.fit(train_ds,validation_data=val_ds,epochs=50,callbacks=[sustabdymas],class_weight=klasiu_svoriai)


nuostolis, tikslumas = modelis.evaluate(test_ds, verbose=1)

print(f"Testo nuostolis: {nuostolis:4f}")
print(f"Testo tikslumas: {tikslumas:4f}")

Klases augmentacija
.

In [ ]:
import shutil
import os

originalas = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\trenyravimas\Tomato___Septoria_leaf_spot"
kopija = r"C:\Users\Vartotojas\Desktop\POMIDORAI\augmented_testas\Tomato___Septoria_leaf_spot"

os.makedirs(os.path.dirname(kopija), exist_ok=True)
shutil.copytree(originalas, kopija, dirs_exist_ok=True)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import os

klases_aplankas_augmentacijai = r"C:\Users\Vartotojas\Desktop\POMIDORAI\augmented_testas\Tomato___Septoria_leaf_spot"
paveiksleliu_generavimas = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

reikalingas_kiekis = 200
sukurta = 0

originalus_failai = [f for f in os.listdir(klases_aplankas_augmentacijai) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

for paveikslelis in originalus_failai:
    naujas_kelias = os.path.join(klases_aplankas_augmentacijai, paveikslelis)
    paveikslelis = load_img(naujas_kelias, target_size=(224, 224))
    paveikslelis_masyvas = img_to_array(paveikslelis)
    paveikslelis_dimensija= paveikslelis_masyvas.reshape((1,) + paveikslelis_masyvas.shape)

    for imtis_batch in paveiksleliu_generavimas.flow(paveikslelis_dimensija, batch_size=1, save_to_dir=klases_aplankas_augmentacijai,
                              save_prefix='aug', save_format='jpg'):
        sukurta += 1
        if sukurta >= reikalingas_kiekis:
            break
    if sukurta >= reikalingas_kiekis:
        break

print(sukurta)

200


In [113]:
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf

dydis = (128, 128)
batch_size = 32
SEED = 42

train_ds = tf.keras.utils.image_dataset_from_directory(
    r"C:\Users\Vartotojas\Desktop\POMIDORAI\augmented_testas",
    image_size=(224, 224),
    batch_size=32,
    label_mode="int",
    # label_mode="categorical",
    shuffle=True,
    seed=42)

val_ds = tf.keras.utils.image_dataset_from_directory(
    r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\validacija",
    image_size=(224, 224),
    batch_size=32,
    label_mode="int",
    # label_mode="categorical",
    shuffle=False,
    seed=42)

test_ds = tf.keras.utils.image_dataset_from_directory(
    r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\testas",
    image_size=(224, 224),
    batch_size=32,
    label_mode="int",
    # label_mode="categorical",
    shuffle=False,
    seed=42)

Found 4697 files belonging to 6 classes.
Found 900 files belonging to 6 classes.
Found 600 files belonging to 6 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping


ivesties_forma = (224,224, 3)
klasiu_skaicius = len(train_ds.class_names)

mobilenet_pagrindas = MobileNetV2(
    input_shape=ivesties_forma,
    include_top=False,
    weights='imagenet')

mobilenet_pagrindas.trainable = False 

modelis = models.Sequential([
    layers.Rescaling(1./255, input_shape=ivesties_forma),
    mobilenet_pagrindas,
    layers.GlobalAveragePooling2D(),
    layers.Dense(180, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(100, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(klasiu_skaicius, activation='softmax')])


modelis.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00008),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

sustabdymas = EarlyStopping(monitor='val_loss',patience=8,restore_best_weights=True)

history = modelis.fit(train_ds,validation_data=val_ds,epochs=70,callbacks=[sustabdymas])


nuostolis, tikslumas = modelis.evaluate(test_ds, verbose=1)

print(f"Testo nuostolis: {nuostolis:4f}")
print(f"Testo tikslumas: {tikslumas:4f}")

Atrakinti sluoksniai 
.

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping


ivesties_forma = (224,224, 3)
klasiu_skaicius = len(train_ds.class_names)

mobilenet_pagrindas = MobileNetV2(
    input_shape=ivesties_forma,
    include_top=False,
    weights='imagenet')

mobilenet_pagrindas.trainable = True


for layer in mobilenet_pagrindas.layers[:-30]:
    layer.trainable = False

modelis = models.Sequential([
    layers.Rescaling(1./255, input_shape=ivesties_forma),
    mobilenet_pagrindas,
    layers.GlobalAveragePooling2D(),
    layers.Dense(180, activation='elu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(100, activation='elu'),
    layers.Dropout(0.3),
    layers.Dense(64, activation='elu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(klasiu_skaicius, activation='softmax')])

modelis.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.000005),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

sustabdymas = EarlyStopping(monitor='val_loss',patience=8,restore_best_weights=True)


history = modelis.fit(train_ds,validation_data=val_ds,epochs=150,callbacks=[sustabdymas])


nuostolis, tikslumas = modelis.evaluate(test_ds, verbose=1)

print(f"Testo nuostolis: {nuostolis:4f}")
print(f"Testo tikslumas: {tikslumas:4f}")
